In [76]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
df = pd.read_csv('complaints_25Nov21.csv')
df.head()
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207260 entries, 0 to 207259
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 207260 non-null  object
 1   Product                       207260 non-null  object
 2   Sub-product                   164245 non-null  object
 3   Issue                         207260 non-null  object
 4   Sub-issue                     10347 non-null   object
 5   Consumer complaint narrative  29391 non-null   object
 6   Company public response       58458 non-null   object
 7   Company                       207260 non-null  object
 8   State                         205066 non-null  object
 9   ZIP code                      197974 non-null  object
 10  Tags                          28265 non-null   object
 11  Consumer consent provided?    51313 non-null   object
 12  Submitted via                 207260 non-null  object
 13 

In [77]:
## convert y-variable to 0s and 1s 
## use label encoder 
le = preprocessing.LabelEncoder()
X = df[['Product', 'Sub-product', 'Issue', 'State', 'Tags', 'Submitted via', 'Company response to consumer', 'Timely response?']].apply(le.fit_transform)
y = le.fit_transform(df['Consumer disputed?'])

In [78]:
from sklearn.model_selection import train_test_split
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
from imblearn.under_sampling import RandomUnderSampler
if y_train.mean() < 0.3:
    undersampler = RandomUnderSampler(random_state=123)
    X_train, y_train = undersampler.fit_resample(X_train, y_train)

In [79]:
## Check what proportion of complaints in  training dataset are disputed.  
## If this proportion is less than 30%, use random undersampling with random_state = 123 to balance dataset. 
from xgboost import XGBClassifier
model_1 = XGBClassifier(random_state=123)
model_1.fit(X_train, y_train)

from sklearn.metrics import classification_report, confusion_matrix
y_pred = model_1.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
basecase_cost = y_test.sum() * 600 + (len(y_test) - y_test.sum()) * 100
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
modelcost = tn * 100 + fp * (100 + 90) + fn * 600 + tp * (100 + 90)
thresholds = np.linspace(0, 1, 100)
for threshold in thresholds:
    y_predthreshold = (model_1.predict_proba(X_test)[:, 1] > threshold).astype(int)

D:\studyresearch\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\studyresearch\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[19:13:02] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.84      0.52      0.64     32504
           1       0.27      0.63      0.38      8948

    accuracy                           0.55     41452
   macro avg       0.55      0.58      0.51     41452
weighted avg       0.71      0.55      0.59     41452

[[17049 15455]
 [ 3315  5633]]


## Question 1

In [81]:
## We need to calculate the number of the disputes, and the total cases in the test set.
## then we can calculate the proprotion which is dipute divided by cases.
numdisputes = sum(y_test)
totalcases = len(y_test)
proportiondisputes = numdisputes / totalcases
print('The proportion of consumers raised a dispute is', proportiondisputes)

The proportion of consumers raised a dispute is 0.21586413200810575


## Question 2

In [82]:
## We resamples the training data to balance the number of disputed and undisputed cases, making sure each category is equally represented. 
## Then calculateing and showing the percentage of disputed cases in this dataset.
from imblearn.under_sampling import RandomUnderSampler
undersampler = RandomUnderSampler(random_state=123)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)
num_disputes = sum(y_train_resampled)
totalcases_resampled = len(y_train_resampled)
proportiondisputesresampled = num_disputes / totalcases_resampled
print('The proportion of consumers in the training dataset is' , proportiondisputesresampled)

The proportion of consumers in the training dataset is 0.5


## Question 3


In [83]:
## We need to use under-sampling to address class imbalance. Then making predictions on the test set and calculating the cases marked as Disputed(YES), 
## Which measure the ability to identidy all actual disputed cases
undersampler = RandomUnderSampler(random_state=123)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)
model_2 = XGBClassifier(random_state=123)
model_2.fit(X_train_resampled, y_train_resampled)
y_pred = model_2.predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)
 # Assuming '1' is'Yes'
recalldisputed_yes = report['1']['recall'] 
print('The recall for the category is',recalldisputed_yes,'it is between 0.6 and 0.67.')

D:\studyresearch\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
D:\studyresearch\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[19:13:32] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
The recall for the category is 0.6295261510952168 it is between 0.6 and 0.67.


## Question 4

In [85]:
cost = len(y_test) * 100
additionalcost = sum(y_test) * 500
# Calculate the total cost
totalcost = cost + additionalcost
print('The total cost to the banks of dealing with the complaints is',totalcost)

The total cost to the banks of dealing with the complaints is 8619200


## Question 5

In [86]:
cost_base = 100  
cost_extra = 90  
cost_dispute = 500  
total_cost1 = ((tn + fp + fn + tp) * cost_base) + ((tp + fp) * cost_extra) + (fn * cost_dispute)
total_cost1
print('Now the cost to the banks of dealing with the complaints in the test set is',total_cost1)

Now the cost to the banks of dealing with the complaints in the test set is 7700620


## Question 6

In [87]:
from sklearn.metrics import confusion_matrix
# Predicted probabilities of the positive class
predicted_probabilities = model_xgb.predict_proba(X_test)[:, 1]
## We need to have the basic cost, dispute cost, investigation cost. Then initializing the variables to track  the best
## threshold values from 0 to 1 to find the minimum total cost.

basic_cost = 100 
investigation_cost = 90 
dispute_cost = 500  
lowest_cost = np.inf  

for threshold_value in np.linspace(0, 1, 101):
    predictions_with_threshold = (predicted_probabilities >= threshold_value).astype(int)
    true_neg, false_pos, false_neg, true_pos = confusion_matrix(y_test, predictions_with_threshold).ravel()
    current_total_cost = (true_neg + false_pos + false_neg + true_pos) * basic_cost + \
                         (false_pos * investigation_cost) + (false_neg * dispute_cost)

    if current_total_cost < lowest_cost:
        lowest_cost = current_total_cost
print("Lowest Total Cost based on the observations in the test set is:", lowest_cost)



Lowest Total Cost based on the observations in the test set is: 6902720


## Question 7

In [88]:
# Obtain probabilities for the positive class
positive_class_probabilities = model_xgb.predict_proba(X_test)[:, 1]

## We need to have the initial handling cost, additional investigation cost, and dispute cost.
initial_handling_cost = 100  
additional_investigation_cost = 90  
dispute_resolution_cost = 500  

## We need to start to make sure any first calculation is lower and also storing the decision yielding the lowest cost
optimal_cost = np.inf 
optimal_decision_threshold = None 

## We want to find the most values by converting probabilities and also understanding the prediction outcomes, and compute
## the total cost, investigationm and dispute costs.
for decision_threshold in np.linspace(0, 1, 101):
    predicted_outcomes = (positive_class_probabilities >= decision_threshold).astype(int)
    true_negative, false_positive, false_negative, true_positive = confusion_matrix(y_test, predicted_outcomes).ravel()
    threshold_total_cost = (true_negative + false_positive) * initial_handling_cost + \
                           false_positive * additional_investigation_cost + \
                           false_negative * dispute_resolution_cost
    if threshold_total_cost < optimal_cost:
        optimal_cost = threshold_total_cost
        optimal_decision_threshold = decision_threshold
print("The lowest dollar cost achieved is:", optimal_decision_threshold)


The lowest dollar cost achieved is: 0.4
